In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interactive, IntSlider, FloatSlider, Dropdown, VBox, HBox, Layout, Checkbox
from IPython.display import display, clear_output
%matplotlib inline 

# --- Fonctions de Génération des Signaux ---

def generate_signal(signal_type, T, fs_cont, f0, duty_cycle=0.5):
    """
    Génère un signal continu (très sur-échantillonné).
    """
    num_samples_cont = max(2, int(T * fs_cont))
    t_cont = np.linspace(0, T, num_samples_cont, endpoint=False)
    
    if signal_type == "Sinus":
        s_cont = np.sin(2 * np.pi * f0 * t_cont)
    elif signal_type == "Carré":
        s_cont = signal.square(2 * np.pi * f0 * t_cont, duty=duty_cycle)
    elif signal_type == "Triangle":
        s_cont = signal.sawtooth(2 * np.pi * f0 * t_cont, width=0.5)
    elif signal_type == "Sinus Multiple":
        s_cont = np.sin(2 * np.pi * f0 * t_cont) + 0.5 * np.sin(2 * np.pi * 3 * f0 * t_cont)
    else: 
        s_cont = np.sin(2 * np.pi * f0 * t_cont)
        
    return t_cont, s_cont

def get_discrete_signal(t_cont, s_cont, T, fs_e):
    """
    Échantillonne le signal continu pour obtenir le signal discret s[n].
    """
    N_e = int(T * fs_e)
    if N_e < 2:
        raise ValueError("Le nombre d'échantillons est trop faible (N < 2). Augmentez Fe ou T.")
        
    indices_e = np.round(np.linspace(0, len(t_cont) - 1, N_e)).astype(int)
    
    t_e = t_cont[indices_e]
    s_e = s_cont[indices_e]
    
    return t_e, s_e, N_e

# --- Fonction de Tracé Interactive et Analyse ---

def plot_dft_analysis(f_e, signal_type, f0, T_max, show_periodicity, show_discrete_spectrum, show_analog_signal):
    """
    Fonction principale. Génère les signaux et affiche les spectres (Continu et TFSD) centrés.
    Le tracé est conditionnel aux trois checkboxes.
    """
    
    # Paramètres fixes
    Fs_cont = 10000    # Fréquence d'échantillonnage pour le signal "continu"
    F_AXIS_LIMIT = 50  # Limite fixe pour l'axe des fréquences (en Hz)

    
    # --- 1. Génération des signaux ---
    
    try:
        t_cont, s_cont = generate_signal(signal_type, T_max, Fs_cont, f0)
        t_e, s_e, N_e = get_discrete_signal(t_cont, s_cont, T_max, f_e)
    except ValueError as e:
        # Affichage d'erreur
        fig, ax = plt.subplots(1, 1, figsize=(12, 5))
        ax.text(0.5, 0.5, f"Erreur : {e}", ha='center', va='center', fontsize=16, color='red')
        ax.axis('off')
        plt.show()
        return

    # Vérification: si rien n'est affiché, avertir l'utilisateur
    if not (show_analog_signal or show_discrete_spectrum):
        fig, ax = plt.subplots(1, 1, figsize=(12, 5))
        ax.text(0.5, 0.5, "Activez au moins une des checkbox d'affichage.", ha='center', va='center', fontsize=16, color='orange')
        ax.axis('off')
        plt.show()
        return


    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # --- Sous-graphe 1 : Domaine Temporel ---
    
    ax1.set_title(f"Domaine Temporel : Signal Continu et Échantillonné (Fe = {f_e:.1f} Hz, T = {T_max:.2f} s)")
    
    # 1. Signal Continu (Analogique) - Tracé conditionnel
    if show_analog_signal:
        ax1.plot(t_cont, s_cont, label='Signal Continu', color='blue', alpha=1)
    
    # 2. Signal discret (Échantillons) - Tracé conditionnel
    if show_discrete_spectrum:
        ax1.stem(t_e, s_e, basefmt=" ", linefmt='r-', markerfmt='ro', label=f'Échantillons Discrets (N={N_e})')
    
    ax1.set_xlabel("Temps (s)")
    ax1.set_ylabel("Amplitude")
    ax1.legend()
    ax1.grid(True)
    ax1.set_xlim(0, T_max)
    
    # --- Sous-graphe 2 : Domaine Fréquentiel ---
    
    # --- 3. Spectre Continu (Référence) ---
    N_cont_initial = len(s_cont)
    Zero_pad_factor_cont = 4
    N_fft_cont = Zero_pad_factor_cont * N_cont_initial
    
    s_cont_padded = np.pad(s_cont, (0, N_fft_cont - N_cont_initial), 'constant')
    S_cont = np.fft.fft(s_cont_padded)
    
    freq_cont = np.fft.fftfreq(N_fft_cont, 1/Fs_cont)
    Spectre_cont_bilateral = np.abs(S_cont) / N_cont_initial 
    
    Spectre_cont_shifted = np.fft.fftshift(Spectre_cont_bilateral)
    freq_cont_shifted = np.fft.fftshift(freq_cont)

    # --- 4. Spectre TFSD (Calcul du motif de base) ---
    N_fft_tfsd = 8 * N_e  
    s_e_padded = np.pad(s_e, (0, N_fft_tfsd - N_e), 'constant')
    S_tfsd_base = np.fft.fft(s_e_padded)

    freq_tfsd_base = np.fft.fftfreq(N_fft_tfsd, 1/f_e)
    Spectre_tfsd_base_bilateral = np.abs(S_tfsd_base) / N_e 
    
    
    # Centrage du motif de base
    Spectre_tfsd_base_shifted = np.fft.fftshift(Spectre_tfsd_base_bilateral)
    freq_tfsd_base_shifted = np.fft.fftshift(freq_tfsd_base)
    
    
    # --- 5. Construction des spectres étendus (pour les répliques) ---
    
    X_label = rf"Fréquence (Hz) [Axe fixe $\pm {F_AXIS_LIMIT} \text{{ Hz}}]$"
        
    num_half_periods = int(np.ceil(F_AXIS_LIMIT / f_e))
    
    freq_list = []
    
    # On itère sur les répliques
    for i in range(-num_half_periods, num_half_periods + 1):
        freq_list.append(freq_tfsd_base_shifted + f_e * i)
        
    freq_tfsd_ext = np.concatenate(freq_list)
    Spectre_tfsd_ext = np.tile(Spectre_tfsd_base_shifted, len(freq_list))
    
    Spectre_cont_to_plot = Spectre_cont_shifted
    freq_cont_to_plot = freq_cont_shifted
    
    # --- Tracés ---
    
    ax2.set_title(rf"Domaine Fréquentiel : Spectre Continu vs. Spectre Échantillonné (Centré, avec $F_e = {f_e:.1f} \text{{ Hz}}$)")
    
    # 1. Spectre Continu (Théorique) - Tracé conditionnel
    if show_analog_signal:
        indices_visibles = (freq_cont_to_plot >= -F_AXIS_LIMIT) & (freq_cont_to_plot <= F_AXIS_LIMIT)

        ax2.plot(freq_cont_to_plot[indices_visibles], 
                 Spectre_cont_to_plot[indices_visibles], 
                 label='Spectre Continu (Théorique)', 
                 color='blue', 
                 linewidth=2)

    
    # 2. Spectre TFSD (Tracé conditionnel)
    if show_discrete_spectrum:
        
        # Tracé périodique (répliques visibles)
        if show_periodicity:
            # Répliques avec alpha=0.5
            indices_tfsd_visibles = (freq_tfsd_ext >= -F_AXIS_LIMIT) & (freq_tfsd_ext <= F_AXIS_LIMIT)
            
            ax2.plot(freq_tfsd_ext[indices_tfsd_visibles], 
                     Spectre_tfsd_ext[indices_tfsd_visibles], 
                     label='Spectre TFSD (Période Fe)', 
                     color='red', 
                     linestyle='-', 
                     linewidth=1.5,
                     alpha=0.5) 
            
        # Tracé du motif de base SEUL dans la fenêtre de Nyquist [-Fe/2, Fe/2]
        indices_tfsd_base = (freq_tfsd_base_shifted >= -f_e/2) & (freq_tfsd_base_shifted <= f_e/2)

        # Tracé du spectre de base (alpha=1.0)
        ax2.plot(freq_tfsd_base_shifted[indices_tfsd_base], 
                 Spectre_tfsd_base_shifted[indices_tfsd_base], 
                 label='Spectre TFSD (Base [-Fe/2, Fe/2])', 
                 color='red', 
                 linestyle='-', 
                 linewidth=2.5, 
                 alpha=0.5)
             
    
    # Lignes de référence
    num_periods_max_ref = int(F_AXIS_LIMIT / f_e) + 1 
    
    for i in np.arange(-num_periods_max_ref, num_periods_max_ref + 1):
        # Multiples de Fe
        ax2.axvline(x=i * f_e, color='gray', linestyle=':', alpha=0.5, zorder=0)
        # Multiples de Fe/2
        ax2.axvline(x=i * f_e + f_e/2, color='orange', linestyle='--', alpha=0.5, zorder=0)

    # Mise en évidence de l'aliasing
    if f0 > f_e / 2:
        # Fréquence réelle (f0)
        if show_analog_signal: # Marquer f0 seulement si le signal analogique est visible
             ax2.axvline(x=f0, color='red', linestyle=':', alpha=0.7, label='Fréquence réelle f0' if f0 <= F_AXIS_LIMIT else "", zorder=1)
             ax2.axvline(x=-f0, color='red', linestyle=':', alpha=0.7, zorder=1)
        
        # Répliques d'aliasing
        if show_discrete_spectrum: # Marquer le repliement seulement si le spectre discret est visible
            for k in range(-num_periods_max_ref, num_periods_max_ref + 1):
                f_replique_pos = f0 - k * f_e
                
                is_aliasing_label = (k == 1 and np.abs(f_replique_pos) <= f_e/2 and f0 > f_e/2)
                
                if np.abs(f_replique_pos) <= F_AXIS_LIMIT:
                     ax2.axvline(x=f_replique_pos, color='purple', linestyle='--', alpha=0.8, 
                                 label='Aliasing/Repliement' if is_aliasing_label else "", zorder=1)

        
    ax2.set_xlabel(X_label)
    ax2.set_ylabel("Magnitude (Normalisée)")
    ax2.set_xlim(-F_AXIS_LIMIT, F_AXIS_LIMIT)
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# --- Définition des Widgets ---

# Définition des widgets de base
layout_style = Layout(width='95%')
T_max_slider = FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='Durée T (s):', continuous_update=False, layout=layout_style)
f0_slider = FloatSlider(min=0.5, max=20.0, step=0.5, value=5.0, description='f0 (Hz):', continuous_update=False, layout=layout_style)
f_e_slider = IntSlider(min=1, max=100, step=1, value=75, description='Fe (Hz):', continuous_update=False, layout=layout_style)
signal_dropdown = Dropdown(options=["Sinus", "Carré", "Triangle", "Sinus Multiple"], value="Sinus", description='Signal:', layout=layout_style)

# Checkboxes
# show_periodicity va être gérée conditionnellement
periodicity_checkbox = Checkbox(value=True, description='Afficher Périodicité (Fe)', layout=Layout(width='95%'))
discrete_spectrum_checkbox = Checkbox(value=True, description='Afficher Signal Échantillonné', layout=Layout(width='95%'))
analog_signal_checkbox = Checkbox(value=True, description='Afficher Signal Analogique', layout=Layout(width='95%'))

# --- Logique de Liaison (Observer) pour la Conditionnalité ---

def toggle_periodicity(change):
    """
    Active ou désactive la checkbox de périodicité en fonction de l'état
    de la checkbox du signal échantillonné.
    """
    is_checked = change['new']
    periodicity_checkbox.disabled = not is_checked
    if not is_checked:
        # Optionnel: décocher périodicité si le signal échantillonné est masqué
        periodicity_checkbox.value = False 

# Ajout de l'observateur au widget du signal échantillonné
discrete_spectrum_checkbox.observe(toggle_periodicity, names='value')

# --- Organisation des Widgets en Deux Colonnes ---

# Colonne 1: Fréquences et options d'affichage
colonne_frequences = VBox([
    f0_slider, 
    f_e_slider,
    analog_signal_checkbox,
    discrete_spectrum_checkbox,
], layout=Layout(width='45%'))

# Colonne 2: Durée, Type de Signal et option de périodicité (maintenant conditionnelle)
colonne_temps_signal = VBox([
    T_max_slider, 
    signal_dropdown,
    periodicity_checkbox, # Déplacé à droite
], layout=Layout(width='55%'))

widgets_ui = HBox([colonne_frequences, colonne_temps_signal])

# --- Création du lien interactif et Affichage ---

# Création du lien interactif (on utilise les noms des arguments de la fonction)
plot_output = interactive(
    plot_dft_analysis,
    f0=f0_slider,
    signal_type=signal_dropdown,
    f_e=f_e_slider,
    T_max=T_max_slider,
    show_periodicity=periodicity_checkbox,
    show_discrete_spectrum=discrete_spectrum_checkbox,
    show_analog_signal=analog_signal_checkbox
)

# Affichage final
display(VBox([widgets_ui, plot_output.children[-1]]))